In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time
import cv2

from joblib import Parallel, delayed

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *

import matplotlib.pyplot as plt
%matplotlib inline

Setting environment for Gordon


In [2]:
from skimage.filters import threshold_otsu
from scipy.ndimage.morphology import distance_transform_edt
from skimage.feature import peak_local_max
from skimage.morphology import watershed, binary_dilation, disk
from skimage.segmentation import mark_boundaries, find_boundaries
from skimage.measure import regionprops, label
from sklearn.cluster import KMeans
from skimage.transform import rotate, resize

In [3]:
stack = 'MD589'

# first_bs_sec, last_bs_sec = section_range_lookup[stack]
# mask = imread('/home/yuncong/csd395/CSHL_data_processed/%s_cellDetectionMask_lossless.png'%stack).astype(np.bool)[...,0]

# bbox_xmin, bbox_ymin, bbox_w, bbox_h = np.array(brainstem_bbox_lookup_midbrain[stack])*32
# bbox_xmax = bbox_xmin + bbox_w - 1
# bbox_ymax = bbox_ymin + bbox_h - 1

In [4]:
min_blob_area = 40
max_blob_area = 1000

In [5]:
output_dir = create_if_not_exists('/home/yuncong/csd395/CSHL_cells_v2/detected_cells/' + stack)

In [ ]:
def draw_cell_contour_on_image(stack, sec, kmeans, bbox, export=False, export_fn=None):
    
#     stack = 'MD589'
#     sec = 113

    region_features = bp.unpack_ndarray_file(output_dir + '/%(stack)s_%(sec)04d_sizeAndHuMoments.bp' % {'stack': stack, 'sec': sec})    
    valid_props = pickle.load(open(output_dir + '%(stack)s_%(sec)04d_blobCoords.pkl' % {'stack': stack, 'sec': sec}, 'r'))

    bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax = bbox
    
    img = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))
    img_gray = rgb2gray(img[bbox_ymin:bbox_ymax+1, bbox_xmin:bbox_xmax+1])

    t = time.time()

    kmeans_labels= kmeans.predict(region_features)
    plt.bar(np.arange(n_clusters), np.bincount(kmeans_labels, minlength=n_clusters));

    sys.stderr.write('kmeans: %.2f\n' % (time.time() - t) )

    t = time.time()

    kmeans_labelmap = np.zeros_like(img_gray, np.int)
    for pr, l in zip(valid_props, kmeans_labels):
        coords = pr.coords
        kmeans_labelmap[coords[:, 0], coords[:, 1]] = l + 1

    sys.stderr.write('labelmap: %.2f\n' % (time.time() - t) )

    t = time.time()
    def f(km_l): return find_boundaries(kmeans_labelmap == km_l)
    bordermaps = Parallel(n_jobs=16)(delayed(f)(km_l) for km_l in range(1, n_clusters+1))
    sys.stderr.write('bordermap: %.2f\n' % (time.time() - t) )

    t = time.time()
    kmeans_bordermap = img_as_ubyte(gray2rgb(img_gray))
    for km_l in range(n_clusters):
        kmeans_bordermap[bordermaps[km_l]] = colors[km_l]
    sys.stderr.write('bordermap: %.2f\n' % (time.time() - t) )
    
    if export:
        if export_fn is None:
            export_fn = '/home/yuncong/csd395/CSHL_cells_kmeansBorderMaps/%(stack)s_%(sec)04d_kmeansBorderMap_sepNorm.jpg' % {'stack':stack, 'sec':sec}
            
        imsave(export_fn, kmeans_bordermap)
    
    return kmeans_bordermap

In [ ]:
display_image(kmeans_bordermap)

In [ ]:
all_sources = []

for sec in range(first_bs_sec, last_bs_sec, 50):    
    blob_orientations = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobOrientations.bp' % {'stack': stack, 'sec': sec})
#     blob_centroids = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobCentroids.bp' % {'stack': stack, 'sec': sec})
#     blob_majorAxisLens = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobMajorAxisLen.bp' % {'stack': stack, 'sec': sec})
#     blob_minorAxisLens = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobMinorAxisLen.bp' % {'stack': stack, 'sec': sec})

#     blob_index = 0
    
#     fitted_ellipse_axisParallel_ys, \
#     fitted_ellipse_axisParallel_xs = ellipse(blob_centroids[blob_index][0], blob_centroids[blob_index][1], 
#                                              blob_majorAxisLens[blob_index], blob_minorAxisLens[blob_index])
    
#     fitted_ellipse_axisParallel_rotated = 
    
#     np.dot(R, np.c_[fitted_ellipse_axisParallel_xs - blob_centroids[blob_index][1], 
#                     fitted_ellipse_axisParallel_ys - blob_centroids[blob_index][0]])
    
    
    n_blobs = len(blob_orientations)
    print n_blobs
    all_sources.append([(sec, i) for i in range(n_blobs)])
    
all_sources = np.concatenate(all_sources)

In [ ]:
len(all_sources)

In [ ]:
n_clusters = 30
colors = np.r_[[(0,0,0)], random_colors(n_clusters)].astype(np.uint8)

In [ ]:
# KMeans on Hu-moment features.

In [ ]:
all_features = []
all_sources = []

for sec in range(first_bs_sec, last_bs_sec, 50):
# for sec in [113]:
    fs = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_features.bp' % {'stack': stack, 'sec': sec})    
    all_sources.append([(sec, i) for i in range(len(fs))])
    all_features.append(fs)    
    
all_features = np.concatenate(all_features)
all_sources = np.concatenate(all_sources)

In [ ]:
t = time.time()

kmeans = KMeans(n_clusters=n_clusters, init='k-means++', n_init=10, max_iter=1000, tol=0.00001,
                precompute_distances='auto', verbose=0, n_jobs=-1)

kmeans.fit(all_features)

sys.stderr.write('kmeans: %.2f\n' % (time.time() - t) ) # 164s - 1145k patches

In [ ]:
# from sklearn.externals import joblib
# joblib.dump(kmeans, '/home/yuncong/csd395/CSHL_cells/kmeans.pkl') 

# kmeans = joblib.load('/home/yuncong/csd395/CSHL_cells/kmeans.pkl') 
# n_clusters = kmeans.n_clusters

In [ ]:
# kmeans label histogram

plt.figure(figsize=(20,5));
bc = np.bincount(kmeans.labels_, minlength=n_clusters)
plt.bar(np.arange(n_clusters)-.4, bc);
plt.xticks(np.arange(n_clusters), np.arange(n_clusters)+1);
plt.xlabel('Kmeans label', fontsize=20);
plt.ylabel('Number of blobs', fontsize=20);

In [ ]:
valid_clusters = np.where(bc > 10)[0] + 1
small_clusters = set(range(1, n_clusters+1)) - set(valid_clusters)

print len(small_clusters), 'clusters with fewer than 5 blobs:', small_clusters
print len(valid_clusters), 'valid clusters:', valid_clusters

In [ ]:
from collections import defaultdict
sampled_indices = defaultdict(list)
for l in valid_clusters:
    all_example_indices = np.where(kmeans.labels_ == l-1)[0]
    random_example_indices = np.random.choice(all_example_indices, min(100, len(all_example_indices)), replace=False)
    for ind in random_example_indices:
        sec, sec_ind = all_sources[ind]
        sampled_indices[sec].append((sec_ind, l))
        
sampled_indices.default_factory = None

In [ ]:
# for i in range(n_clusters):
#     create_if_not_exists('/home/yuncong/csd395/CSHL_cells/gallery/%02d' % i)

In [ ]:
margin = 10

In [ ]:
# patches_allLabels_debug = defaultdict(list)
patches_allLabels = defaultdict(list)

for sec, secInd_label_tuples in sampled_indices.iteritems():
    
#     if sec != 93:
#         continue
    
    print sec

    blob_coords = pickle.load(open('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobCoords.pkl' % {'stack': stack, 'sec': sec}, 'r'))
    blob_orientations = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobOrientations.bp' % {'stack': stack, 'sec': sec})
    blob_centers = [coords.mean(axis=0) for coords in blob_coords]
    
    blob_majorAxisLens = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobMajorAxisLen.bp' % {'stack': stack, 'sec': sec}) = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobOrientations.bp' % {'stack': stack, 'sec': sec})
    blob_minorAxisLens = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobMinorAxisLen.bp' % {'stack': stack, 'sec': sec}) = bp.unpack_ndarray_file('/home/yuncong/csd395/CSHL_cells/%(stack)s_%(sec)04d_blobOrientations.bp' % {'stack': stack, 'sec': sec})
    
    img = imread(DataManager.get_image_filepath(stack=stack, section=sec, version='rgb-jpg'))
    img_rgb = img[bbox_ymin:bbox_ymax+1, bbox_xmin:bbox_xmax+1]

    for blob_index_inSec, l in secInd_label_tuples:
        coords = blob_coords[blob_index_inSec]
        p_ymin, p_xmin = coords.min(axis=0)
        p_ymax, p_xmax = coords.max(axis=0)
#         patch = img_rgb[p_ymin:p_ymax+1, p_xmin:p_xmax+1].copy()

        if p_ymin < margin or p_xmin < margin:
            continue

        patch = img_rgb[p_ymin-margin:p_ymax+1+margin, p_xmin-margin:p_xmax+1+margin].copy()
        
        filled = np.zeros(patch.shape[:2])
        filled[coords[:,0]-(p_ymin-margin), coords[:,1]-(p_xmin-margin)] = 1
        boundaries = find_boundaries(filled.astype(np.uint8), mode='inner')
        patch[boundaries] = (255,0,0)
        
#         patch[coords[:,0]-ymin, coords[:,1]-xmin] = (0,0,0)
#         patches_allLabels_debug[l].append([p_ymin, p_ymax, p_xmin, p_xmax])
        
        patch_rotationNormalized = rotate(patch, angle=-np.rad2deg(blob_orientations[blob_index_inSec]), 
                                          center=(blob_centers[blob_index_inSec][1] - (p_xmin-margin), 
                                                  blob_centers[blob_index_inSec][0] - (p_ymin-margin)),
                                          cval=1.)
        
        patches_allLabels[l].append(patch_rotationNormalized)
#         cv2.imwrite('/home/yuncong/csd395/CSHL_cells/gallery/%(label)s/%(sec)04d_%(ind)d.jpg' % {'label':l, 'sec': sec, 'ind': blob_index_inSec}, patch)

patches_allLabels.default_factory = None
# patches_allLabels_debug.default_factory = None

In [ ]:
for l, patches in patches_allLabels.iteritems():
    print 'kmeans label', l
    display_images_in_grids(patches, nc=10)

In [ ]:
! rm /home/yuncong/csd395/CSHL_cells/gallery/gridplots/*

In [ ]:
for l, patches in patches_allLabels.iteritems():
    display_images_in_grids(patches, nc=10, 
            export_fn='/home/yuncong/csd395/CSHL_cells/gallery/gridplots/gridplots_%02d_%d.png'%(l, len(patches)))